<a href="https://colab.research.google.com/github/sayarghoshroy/Hate-Speech-Detection/blob/master/perspective_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Testing Effectiveness of Perspective API features

In [2]:
import pickle
import numpy as np
import random
from sklearn.metrics import classification_report
import pandas as pd
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, neighbors
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Loading datasets

en_load = '/content/drive/My Drive/HASOC_raw_data/perspective_train/en.pickle'
en_pers = {}

ge_load = '/content/drive/My Drive/HASOC_raw_data/perspective_train/ge.pickle'
ge_pers = {}

en_data_load = '/content/drive/My Drive/2020_processed_data/en.pickle'
en_data = {}

ge_data_load = '/content/drive/My Drive/2020_processed_data/ge.pickle'
ge_data = {}

with open(en_load, 'rb') as f:
  en_pers = pickle.load(f)

with open(ge_load, 'rb') as f:
  ge_pers = pickle.load(f)

with open(en_data_load, 'rb') as f:
  en_data = pickle.load(f)

with open(ge_data_load, 'rb') as f:
  ge_data = pickle.load(f)

In [5]:
# Visualizing Data
for key in ge_pers.keys():
  print(str(key))
  # Uncomment to visualize actual values
  # print(str(key) + ": " + str(ge_pers[key]))

TOXICITY_WHOLE
TOXICITY_RAW
SEVERE_TOXICITY_WHOLE
SEVERE_TOXICITY_RAW
IDENTITY_ATTACK_WHOLE
IDENTITY_ATTACK_RAW
INSULT_WHOLE
INSULT_RAW
PROFANITY_WHOLE
PROFANITY_RAW
THREAT_WHOLE
THREAT_RAW


In [6]:
# Testing Load Correctness
ge_data.keys()

dict_keys(['tweet_id', 'task_1', 'task_2', 'hasoc_id', 'full_tweet', 'tweet_raw_text', 'hashtags', 'smiley', 'emoji', 'url', 'mentions', 'numerals', 'reserved_word', 'emotext', 'segmented_hash'])

In [7]:
x = []
y = []

# Uncomment to Run for English
# language = "EN"

# Uncomment to Run for German
language = "GE"

if language == "EN":
    data_size = len(en_data['task_1'])

    for idx in range(data_size):
      y.append(en_data['task_1'][idx])

    x_matrix = []
    for key in en_pers.keys():
      if key == "RAW_SPAN" or key == "WHOLE_SPAN":
        continue
      x_matrix.append(en_pers[key])

    x_matrix = np.asmatrix(x_matrix).T
    y = np.asmatrix(y).T
    # View Dimensions
    print("x Dimensions: " + str(np.shape(x_matrix)))
    print("y Dimensions: " + str(np.shape(y)))

elif language == "GE":
    data_size = len(ge_data['task_1'])

    for idx in range(data_size):
      y.append(ge_data['task_1'][idx])

    x_matrix = []
    for key in ge_pers.keys():
      if key == "RAW_SPAN" or key == "WHOLE_SPAN":
        continue
      x_matrix.append(ge_pers[key])

    x_matrix = np.asmatrix(x_matrix).T
    y = np.asmatrix(y).T
    # View Dimensions
    print("x Dimensions: " + str(np.shape(x_matrix)))
    print("y Dimensions: " + str(np.shape(y)))

x Dimensions: (2373, 12)
y Dimensions: (2373, 1)


In [8]:
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(x_matrix, y, random_state = 42, test_size = 0.2)
train_Y = np.ravel(train_Y)
test_Y = np.ravel(test_Y)

In [9]:
# Viewing Data Shapes
print("Train X: " + str(np.shape(train_X)))
print("Train Y: " + str(np.shape(train_Y)))
print("Test X: " + str(np.shape(test_X)))
print("Test Y: " + str(np.shape(test_Y)))

Train X: (1898, 12)
Train Y: (1898,)
Test X: (475, 12)
Test Y: (475,)


In [10]:
# Sklearn Implementation of SGD with 1e6 iterations
def train(X, y, active = 'relu', sol = 'adam', learn = 'adaptive'):
    reg = MLPClassifier(alpha = 0,
                       learning_rate_init = 1e-2 * 5,
                       learning_rate = learn,
                       hidden_layer_sizes = (32, 64, 128, 256, 512, 256, 64, 32, 16, 8, 4, 1),
                       random_state = 2020,
                       activation = active,
                       max_iter = int(1e5),
                       solver = sol,
                       momentum = 0.9)
    reg.fit(X, y)
    return reg
    # reg is our trained model

def get_train_test_res(reg):
    pred_y_train = reg.predict(train_X)
    pred_y_test = reg.predict(test_X)
    target_names = ['HOF', 'NOT']

    total = 0
    matches = 0
    size = np.shape(pred_y_test)[0]
    for idx in range(size):
      if pred_y_test[idx] == test_Y[idx]:
        matches += 1
      total += 1
    print("Accuracy = " + str(matches / total * 100) + "%")
    print("")
    # Uncomment to View Full Reports
    # print(classification_report(test_Y, pred_y_test, target_names = target_names))

In [11]:
activations = ['tanh'] # , 'relu', 'identity', 'logistic']
solvers = ['adam', 'lbfgs', 'sgd']
learning_rates = ['constant', 'invscaling', 'adaptive']
model_number = 1
# On German, identity activation + sgd solver gives overflow problems
for active in activations:
    print("### " + str(active) + " Activation")
    for sol in solvers:
        if active == 'tanh' and sol == 'lbfgs':
          continue
        if language == "GE" and active == "identity" and sol == "sgd":
          continue
        print("#### " + str(sol) + " Solver")
        for learn in learning_rates:
            model = train(train_X, train_Y, active, sol, learn)
            print(str(model_number) + ". " + " Learn Rate = " + str(learn) + " : ")
            get_train_test_res(model)
            model_number += 1

### tanh Activation
#### adam Solver
1.  Learn Rate = constant : 
Accuracy = 70.94736842105263%

2.  Learn Rate = invscaling : 
Accuracy = 70.94736842105263%

3.  Learn Rate = adaptive : 
Accuracy = 70.94736842105263%

#### sgd Solver
4.  Learn Rate = constant : 
Accuracy = 81.89473684210526%

5.  Learn Rate = invscaling : 
Accuracy = 70.94736842105263%

6.  Learn Rate = adaptive : 
Accuracy = 84.42105263157896%



### Random Seed Used: 42
#### For all experiments

#Results for English

## identity Activation
### lbfgs solver
1. Learn Rate = constant : Accuracy = 90.02695417789758%

2. Learn Rate = invscaling : Accuracy = 90.02695417789758%

3. Learn Rate = adaptive : Accuracy = 90.02695417789758%

### sgd Solver
4. Learn Rate = constant : Accuracy = 89.75741239892183%

5. Learn Rate = invscaling : Accuracy = 88.27493261455525%

6. Learn Rate = adaptive : Accuracy = 89.35309973045821%

### adam Solver
7. Learn Rate = constant : Accuracy = 82.61455525606469%

8. Learn Rate = invscaling : Accuracy = 82.61455525606469%

9. Learn Rate = adaptive : Accuracy = 82.61455525606469%

## logistic Activation
### lbfgs Solver
1. Learn Rate = constant : Accuracy = 47.03504043126684%

2. Learn Rate = invscaling : Accuracy = 47.03504043126684%

3. Learn Rate = adaptive : Accuracy = 47.03504043126684%

### sgd Solver
4. Learn Rate = constant : Accuracy = 47.03504043126684%

5. Learn Rate = invscaling : Accuracy = 52.96495956873315%

6. Learn Rate = adaptive : Accuracy = 47.03504043126684%

### adam Solver
7. Learn Rate = constant : Accuracy = 47.03504043126684%

8. Learn Rate = invscaling : Accuracy = 47.03504043126684%

9. Learn Rate = adaptive : Accuracy = 47.03504043126684%

## relu Activation
### sgd Solver
1. Learn Rate = constant : Accuracy = 89.35309973045821%

2. Learn Rate = invscaling : Accuracy = 84.50134770889488%

3. Learn Rate = adaptive : Accuracy = 89.08355795148249%

### adam Solver
4. Learn Rate = constant : Accuracy = 47.03504043126684%

5. Learn Rate = invscaling : Accuracy = 47.03504043126684%

6. Learn Rate = adaptive : Accuracy = 47.03504043126684%

### lbfgs solver
7. Learn Rate = constant : Accuracy = 88.94878706199461%

8. Learn Rate = invscaling : Accuracy = 88.94878706199461%

9. Learn Rate = adaptive : Accuracy = 88.94878706199461%

## tanh Activation
### sgd Solver
1. Learn Rate = constant : Accuracy = 89.8921832884097%

2. Learn Rate = invscaling : Accuracy = 89.4878706199461%

3. Learn Rate = adaptive : Accuracy = 87.87061994609164%

### adam Solver
4. Learn Rate = constant : Accuracy = 47.03504043126684%

5. Learn Rate = invscaling : Accuracy = 47.03504043126684%

6. Learn Rate = adaptive : Accuracy = 47.03504043126684%

# Results for German

### tanh Activation
#### sgd Solver
1.  Learn Rate = constant : 
Accuracy = 81.89473684210526%

2.  Learn Rate = invscaling : 
Accuracy = 70.94736842105263%

3.  Learn Rate = adaptive : 
Accuracy = 84.42105263157896%

#### adam Solver
4.  Learn Rate = constant : 
Accuracy = 70.94736842105263%

5.  Learn Rate = invscaling : 
Accuracy = 70.94736842105263%

6.  Learn Rate = adaptive : 
Accuracy = 70.94736842105263%

### logistic Activation
#### sgd Solver
1.  Learn Rate = constant : 
Accuracy = 70.94736842105263%

2.  Learn Rate = invscaling : 
Accuracy = 70.94736842105263%

3.  Learn Rate = adaptive : 
Accuracy = 70.94736842105263%

#### adam Solver
4.  Learn Rate = constant : 
Accuracy = 70.94736842105263%

5.  Learn Rate = invscaling : 
Accuracy = 70.94736842105263%

6.  Learn Rate = adaptive : 
Accuracy = 70.94736842105263%

#### lbfgs Solver
7.  Learn Rate = constant : 
Accuracy = 70.94736842105263%

8.  Learn Rate = invscaling : 
Accuracy = 70.94736842105263%

9.  Learn Rate = adaptive : 
Accuracy = 70.94736842105263%

### relu Activation
#### sgd Solver
1.  Learn Rate = constant : 
Accuracy = 70.94736842105263%

2.  Learn Rate = invscaling : 
Accuracy = 70.94736842105263%

3.  Learn Rate = adaptive : 
Accuracy = 70.94736842105263%

#### adam Solver
4.  Learn Rate = constant : 
Accuracy = 70.94736842105263%

5.  Learn Rate = invscaling : 
Accuracy = 70.94736842105263%

6.  Learn Rate = adaptive : 
Accuracy = 70.94736842105263%

#### lbfgs Solver
7.  Learn Rate = constant : 
Accuracy = 70.94736842105263%

8.  Learn Rate = invscaling : 
Accuracy = 70.94736842105263%

9.  Learn Rate = adaptive : 
Accuracy = 70.94736842105263%

### identity Activation
#### adam Solver
1.  Learn Rate = constant : 
Accuracy = 84.0%

2.  Learn Rate = invscaling : 
Accuracy = 84.0%

3.  Learn Rate = adaptive : 
Accuracy = 84.0%

#### lbfgs Solver
4.  Learn Rate = constant : 
Accuracy = 82.94736842105263%

5.  Learn Rate = invscaling : 
Accuracy = 82.94736842105263%

6.  Learn Rate = adaptive : 
Accuracy = 82.94736842105263%

In [12]:
# ^_^ Thank You